In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from xgboost import XGBClassifier, plot_importance
import shutil
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import seaborn as sns
from sklearn.metrics import confusion_matrix
import pickle
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree




In [2]:
sns.set_palette('colorblind')
from matplotlib.pyplot import tight_layout
# ##SETTING PARAMS FOR MATPLOTLIB FIGURES
plt.rcParams.update({"figure.figsize": (6, 6),
                 "axes.facecolor": "white",
                 "axes.edgecolor": "black"})
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=sns.color_palette('colorblind'))
##set font size
font = {'family': 'sans-serif',
       'weight': 'normal',
       'size': 14}
plt.rc('font', **font)
# ##PANDAS PLOTTING
pd.plotting.register_matplotlib_converters()

In [3]:
merged_glass= "/home/paulharford/college/project/project_data/processed/WEATHERED_merged_v2.csv"
full_path_glass = os.path.abspath(merged_glass)

In [4]:
merged_glass = pd.read_csv(full_path_glass)

In [5]:
merged_glass.head(5)

,region,date,age_group,gender,hip_fracture_count,weather_event,warning_phenomenon,warning_severity,warning_severity_numeric,counties_in_region,...,red_warning_lag,cold_weather,wind_weather,precipitation,heat_weather,no_adverse_weather,cold_lag,wind_lag,precip_lag,heat_lag
0,HSE Dublin and Midlands,2014-01-02,65-69,Female,1.0,1.0,Wind,Yellow,1.0,7,...,0.0,0,1,0,0,0,0.0,0.0,0.0,0.0
1,HSE Dublin and Midlands,2014-01-03,60-64,Female,0.0,1.0,Wind,Yellow,1.0,7,...,0.0,0,1,0,0,0,0.0,1.0,0.0,0.0
2,HSE Dublin and Midlands,2014-01-03,65-69,Female,0.0,1.0,Wind,Yellow,1.0,7,...,0.0,0,1,0,0,0,0.0,1.0,0.0,0.0
3,HSE Dublin and Midlands,2014-01-03,70-74,Female,0.0,1.0,Wind,Yellow,1.0,7,...,0.0,0,1,0,0,0,0.0,1.0,0.0,0.0
4,HSE Dublin and Midlands,2014-01-03,75-79,Female,0.0,1.0,Wind,Yellow,1.0,7,...,0.0,0,1,0,0,0,0.0,1.0,0.0,0.0


In [6]:
#import warnings
## Suppress all FutureWarning messages
#warnings.simplefilter(action='ignore', category=FutureWarning)

pathdic={}
pathdic['project_root'] = '/home/paulharford/college/project/ul_project_Msc_AI/analysis/processed_data/analysed_data/glassboost'
pathdic['sprint_root'] = os.getcwd()
###CREATE DATASET AND OUTPUTS DIRECTORIES IF NOT EXISTING
mdirs = ['dataset', 'outputs', 'results']
for mdir in mdirs:
	if not os.path.exists(os.path.join(pathdic['project_root'], mdir)):
		os.makedirs(os.path.join(pathdic['project_root'] , mdir))
		pathdic[mdir]=os.path.join(pathdic['project_root'], mdir)
	else:
		pathdic[mdir]=os.path.join(pathdic['project_root'], mdir)
        
def create_directory(fname):
	"""
	Create a folder if it doesn't already exist.
	:param fname:path and directory name
	:return:
	"""
	if not os.path.exists(fname):
		os.makedirs(fname)

def create_single_subdir(fname=None, path_to_f=None):
	npth = os.path.join(path_to_f, fname)
	if not os.path.exists(npth):
		os.makedirs(npth)
	return npth

def create_single_results_subdir(fname=None, path_to_f=None):
	npth = os.path.join(path_to_f, fname)
	try:
		##DELETE THE FOLDER
		shutil.rmtree(npth)
		os.makedirs(npth)
	except WindowsError:
		os.makedirs(npth)
	return npth

def create_multi_dirs(fname=None, pathdic=None, listdirs=None):
	for d in listdirs:
		###create the dir
		create_directory(os.path.join(pathdic[d], fname))
		if 'figures' in d:
			###update pathdic
			pathdic['savefig'] = os.path.join(pathdic[d], fname)
		elif 'dfs' in d:
			pathdic['savedf'] = os.path.join(pathdic[d], fname)
		else:
			pathdic[d] = os.path.join(pathdic[d], fname)
	return pathdic

def split_long_names(name):
	split_pos = name.rfind(' ', 0, 25)
	if split_pos != -1:
		return name[:split_pos] + '\n' + name[split_pos + 1:]
	else:
		return name

def round_up(numerator, denominator):
	return np.ceil(numerator/denominator) * denominator


def apply_min_ticks(ax, min_ticks=6):
	ax.xaxis.set_major_locator(ticker.MaxNLocator(nbins=min_ticks, prune=None))
	ax.yaxis.set_major_locator(ticker.MaxNLocator(nbins=min_ticks, prune=None))

def specificity_score(y_true, y_pred):
	tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
	return tn / (tn + fp) if (tn + fp) > 0 else 0.0

def img_from_df(df=None, path_out=None, filename=None):
	df = df.map(lambda x: f"{x:.4g}" if isinstance(x, (int, float)) else x)
	####FORMAT INDEX FROM 1 TO N
	df.index = range(1, len(df.index) + 1)
	df.index.name = 'index'
	###ENSURE INDEX IS PRINTED
	df.reset_index(inplace=True)
	fig, ax = plt.subplots(figsize=(df.shape[1] * 2, df.shape[0] * 0.25))
	ax.axis('off')

	table = ax.table(cellText=df.values, colLabels=df.columns, loc='center', cellLoc='center')
	table.auto_set_font_size(False)
	table.set_fontsize(12)
	table.auto_set_column_width(col=list(range(len(df.columns))))
	table.scale(1.2, 1.2)

	plt.savefig(os.path.join(path_out, f'{filename}.png'), bbox_inches='tight', dpi=300)
	plt.close()



In [7]:
def run_cart_gridsearch(X=None, y=None,
                        importance_df=None,
                        top_n_list=None,
                        depth_list=None,
                        path_out=None,
                        label=None):
    if top_n_list is None:
        top_n_list = [7, 9, 13, 15]
    if depth_list is None:
        depth_list = [5, 6, 7]
    
    cart_results = []
    top_features_sorted = list(importance_df.sort_values('Gain', ascending=False)['Feature'])
    
    for n in top_n_list:
        if n > len(top_features_sorted):
            continue
        selected_feats = top_features_sorted[:n]
        X_top = X[selected_feats]
        
        for depth in depth_list:
            X_train, X_test, y_train, y_test = train_test_split(X_top, y, test_size=0.2, stratify=y, random_state=42)
            cart = DecisionTreeClassifier(max_depth=depth, random_state=42)
            cart.fit(X_train, y_train)
            y_pred = cart.predict(X_test)
            
            ##SAVE CART MODELS
            model_filename = f'cart_model_{label}_top{n}_depth{depth}.pkl'
            with open(os.path.join(path_out, model_filename), 'wb') as f:
                pickle.dump(cart, f)
            
            acc = accuracy_score(y_test, y_pred)
            prec = precision_score(y_test, y_pred)
            rec = recall_score(y_test, y_pred)
            spec = specificity_score(y_test, y_pred)
            
            cart_results.append({
                'label': label,
                'top_n_features': n,
                'max_depth': depth,
                'accuracy': acc,
                'precision': prec,
                'recall': rec,
                'specificity': spec,
            })
    
    # Create DataFrame from results
    cart_df = pd.DataFrame(cart_results)
    
    # Add debugging
    print(f"cart_df shape: {cart_df.shape}")
    print(f"cart_df columns: {cart_df.columns.tolist()}")
    if cart_df.empty:
        print("WARNING: cart_df is empty!")
        print(f"top_n_list: {top_n_list}")
        print(f"depth_list: {depth_list}")
        print(f"importance_df shape: {importance_df.shape}")
        print(f"Number of features available: {len(top_features_sorted)}")
        return cart_df  # Return early if empty
    
    # Save CSV
    cart_df.to_csv(os.path.join(path_out, f'cart_results_{label}.csv'), index=False)
    
    ###CREATE IMAGE OF THE GRID RESULTS
    img_from_df(df=cart_df,
                path_out=path_out,
                filename=f'cart_results_{label}')
    
    ###HEATMAP
    # Heatmaps
    for metric, cmap in zip(['accuracy', 'precision', 'recall', 'specificity'],
                            ['YlGnBu', 'Oranges', 'Purples', 'Blues']):
        pivot = cart_df.pivot(index='max_depth', columns='top_n_features', values=metric)
        plt.figure(figsize=(6, 6), constrained_layout=True)
        sns.heatmap(pivot, annot=True, fmt=".2f", cmap=cmap, cbar_kws={'label': metric})
        plt.title(f"CART {metric.capitalize()} Heatmap - {label}")
        plt.xlabel("Top n Features")
        plt.ylabel("Max Depth")
        plt.savefig(os.path.join(path_out, f'xgb_heatmap_{metric}_{label}.png'), dpi=300)
        plt.close()
    
    return cart_df

In [11]:
def run_xgboost_gridsearch(X=None,
						   y=None,
						   param_grid=None,
						   path_out=None,
						   metric=None,
						   label='default'):
	if X is None or y is None or X.shape[0] == 0:
		raise ValueError("Input features X or target y are empty or undefined.")

	xgb_grid_results = []

	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

	for max_depth, n_estimators in product(param_grid['max_depth'], param_grid['n_estimators']):
		model = XGBClassifier(
			#use_label_encoder=False,
			eval_metric='logloss',
			max_depth=max_depth,
			n_estimators=n_estimators,
			random_state=42
		)
		model.fit(X_train, y_train)
		y_pred = model.predict(X_test)
		spec = specificity_score(y_test, y_pred)

		xgb_grid_results.append({
			'filter': label,
			'max_depth': max_depth,
			'n_estimators': n_estimators,
			'accuracy': accuracy_score(y_test, y_pred),
			'precision': precision_score(y_test, y_pred),
			'recall': recall_score(y_test, y_pred),
			'specificity':spec,
		})

	# Save grid results
	xgb_df = pd.DataFrame(xgb_grid_results)
	xgb_df.to_csv(os.path.join(path_out, f'xgb_grid_results_{label}.csv'), index=False)

	###CREATE IMAGE OF THE GRID RESULTS
	img_from_df(df=xgb_df,
				path_out=path_out,
				filename='xgb_grid_results_table')

	# Heatmaps
	for metric, cmap in zip(['accuracy', 'precision', 'recall', 'specificity'], ['YlGnBu', 'Oranges', 'Purples', 'Blues']):
		pivot = xgb_df.pivot(index='max_depth', columns='n_estimators', values=metric)
		plt.figure(figsize=(6, 6), constrained_layout=True)
		sns.heatmap(pivot, annot=True, fmt=".2f", cmap=cmap, cbar_kws={'label': metric})
		plt.title(f"XGBoost {metric.capitalize()} Heatmap - {label}")
		plt.xlabel("n_estimators")
		plt.ylabel("max_depth")
		plt.savefig(os.path.join(path_out, f'xgb_heatmap_{metric}_{label}.png'), dpi=300)
		plt.close()

	# Train best model
	best = xgb_df.sort_values(by=metric, ascending=False).iloc[0]
	best_model = XGBClassifier(
		#use_label_encoder=False,
		eval_metric='logloss',
		max_depth=int(best['max_depth']),
		n_estimators=int(best['n_estimators']),
		random_state=42
	)
	best_model.fit(X_train, y_train)

	# SHAP
	explainer = shap.Explainer(best_model)
	shap_values = explainer(X_test)
	shap.plots.beeswarm(shap_values, max_display=15, order=shap_values.abs.max(0), show=False)
	plt.savefig(os.path.join(path_out, f'SHAP_best_{label}.png'), dpi=300, bbox_inches='tight')
	plt.close()

	# Feature importance
	booster = best_model.get_booster()
	fig, ax = plt.subplots(figsize=(12, 8))
	plot_importance(booster, importance_type='gain', max_num_features=15, ax=ax)
	plt.title(f"Top Features by Gain (Best XGBoost) - {label}")
	plt.savefig(os.path.join(path_out, f'top_features_by_gain_best_{label}.png'), dpi=300, bbox_inches='tight')
	plt.close()

	importance_df = booster.get_score(importance_type='gain')
	importance_df_df = pd.DataFrame(list(importance_df.items()), columns=['Feature', 'Gain'])
	importance_df_df = importance_df_df.sort_values(by='Gain', ascending=False)
	importance_df_df['Cumulative Gain'] = importance_df_df['Gain'].cumsum()
	importance_df_df['Cumulative Gain %'] = importance_df_df['Cumulative Gain'] / importance_df_df['Gain'].sum() *100
	importance_df_df.to_csv(os.path.join(path_out, 'importance_df.csv'), index=False)

	###CREATE IMAGE OF THE IMPORTANCE TABLE
	img_from_df(df=importance_df_df,
				path_out=path_out,
				filename='importance_df_table')

	return best_model, importance_df_df
    
print(f"\nXGBoost model trained successfully")
print(f"importance_df shape: {importance_df.shape}")
print(f"importance_df columns: {importance_df.columns.tolist()}")
print(f"Number of features with importance: {len(importance_df)}")
print("\nTop 10 important features:")
print(importance_df.head(10))

# Check if we have enough features
available_features = len(importance_df)
print(f"\nAvailable features: {available_features}")
print(f"Requested top_n_list: [7, 9, 13, 15]")
if available_features < 15:
    print(f"WARNING: Only {available_features} features available, but requesting up to 15")



XGBoost model trained successfully
importance_df shape: (6, 4)
importance_df columns: ['Feature', 'Gain', 'Cumulative Gain', 'Cumulative Gain %']
Number of features with importance: 6

Top 10 important features:
      Feature         Gain  Cumulative Gain  Cumulative Gain %
4       total  1655.493042      1655.493042          35.416351
3        male  1111.912964      2767.406006          59.203765
5  population   919.013672      3686.419678          78.864440
2      female   510.558655      4196.978333          89.786941
0   age_group   410.789062      4607.767395          98.575047
1      gender    66.607628      4674.375023         100.000000

Available features: 6
Requested top_n_list: [7, 9, 13, 15]


In [9]:
X = merged_glass.drop('hip_fracture_count', axis=1)
y = merged_glass['hip_fracture_count']

# Binary classification: fracture vs no fracture
y_binary = (y > 0).astype(int)

print("Columns in X:")
print(X.columns.tolist())
print("\nData types:")
print(X.dtypes)

# Option 1: Drop the date column and encode everything else
X_processed = X.copy()

# Drop date column if it exists
if 'date' in X_processed.columns:
    X_processed = X_processed.drop('date', axis=1)

# Check for any remaining object columns
object_cols = X_processed.select_dtypes(include=['object']).columns
print(f"\nObject columns to encode: {object_cols.tolist()}")

# Encode all object columns
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for col in object_cols:
    le = LabelEncoder()
    X_processed[col] = le.fit_transform(X_processed[col])
    label_encoders[col] = le

# Verify all columns are numeric
print("\nData types after processing:")
print(X_processed.dtypes)
print(f"\nAny remaining object columns: {X_processed.select_dtypes(include=['object'])}")

dfu = merged_glass[merged_glass['hip_fracture_count'].isna()]

Columns in X:
['region', 'date', 'age_group', 'gender', 'weather_event', 'warning_phenomenon', 'warning_severity', 'warning_severity_numeric', 'counties_in_region', 'county_weight', 'female', 'male', 'total', 'population', 'log_population', 'month', 'season', 'is_winter', 'is_spring', 'is_summer', 'is_autumn', 'orange_warning', 'red_warning', 'yellow_warning', 'orange_warning_lag', 'red_warning_lag', 'cold_weather', 'wind_weather', 'precipitation', 'heat_weather', 'no_adverse_weather', 'cold_lag', 'wind_lag', 'precip_lag', 'heat_lag']

Data types:
region                       object
date                         object
age_group                    object
gender                       object
weather_event               float64
warning_phenomenon           object
warning_severity             object
warning_severity_numeric    float64
counties_in_region            int64
county_weight               float64
female                      float64
male                        float64
total         

In [15]:
############
### MODELLING ASSESSMENT
##########
###RUN XGBOOST ASSESSMENT
### XGBoost hyperparameter grid
param_grid = {
    'max_depth': [5, 6, 10, 20],
    'n_estimators': [50, 100, 150]
}
metric = 'specificity'

##RUN XGBOOST AND GET BEST MODEL
best_model, importance_df = run_xgboost_gridsearch(X=X_processed,
                                                   y=y_binary,
                                                   param_grid=param_grid,
                                                   metric=metric,
                                                   path_out=pathdic['results'],
                                                   label='dfk')

# Debug XGBoost output
print(f"\nXGBoost model trained successfully")
print(f"importance_df shape: {importance_df.shape}")
print(f"Number of features with importance: {len(importance_df)}")
print("\nTop features:")
print(importance_df)

###save best model to pkl
pickle.dump(best_model,
            open(os.path.join(pathdic['results'], 'XGB_best_model.pkl'), 'wb'))

# CRITICAL FIX: Adjust top_n_list based on available features
available_features = len(importance_df)
if available_features <= 6:
    # For 6 or fewer features, use a more appropriate range
    top_n_list = [3, 4, 5, 6][:available_features]  # This gives [3, 4, 5, 6] for 6 features
else:
    # Original list for datasets with more features
    top_n_list = [7, 9, 13, 15]

# Filter to valid values only
valid_top_n_list = [n for n in top_n_list if n <= available_features]
print(f"\nAdjusted top_n_list from [7, 9, 13, 15] to {valid_top_n_list}")

###BASED ON THE BEST XGBOOST MODEL RUN A CART ANALYSIS
cart_df = run_cart_gridsearch(X=X_processed,
                              y=y_binary,
                              importance_df=importance_df,
                              top_n_list=valid_top_n_list,  # Use the adjusted list
                              depth_list=[5, 6, 7],
                              path_out=pathdic['results'],
                              label='dfk')

# Continue with the rest of your code...
if cart_df.empty:
    print("ERROR: cart_df is still empty after adjustment!")
else:
    print(f"\nCART grid search completed. Generated {len(cart_df)} models.")
    
    best_cart = cart_df.sort_values(
        by=[metric, 'top_n_features', 'max_depth'],
        ascending=[False, True, True]
    ).iloc[0]
    print("\nBest efficient CART model:")
    print(best_cart)
    
filename = f"cart_model_{best_cart['label']}_top{int(best_cart['top_n_features'])}_depth{int(best_cart['max_depth'])}.pkl"
filepath = os.path.join(pathdic['results'], filename)

# Load that specific CART model
with open(filepath, 'rb') as f:
    cart_model = pickle.load(f)

###GET THE TOP FEATURES ONLY
selected_feats = importance_df.sort_values(
	'Gain',
	ascending=False).head(int(best_cart['top_n_features']))['Feature'].tolist()
X_u = dfu[selected_feats]

###PLOT THE  BEST MODEL
fig, ax = plt.subplots(figsize=(20, 10))
plot_tree(cart_model, feature_names=selected_feats, class_names=True, filled=True, max_depth=int(best_cart['max_depth']))
plt.title(f"CART Tree - Top {int(best_cart['top_n_features'])} Features - Depth {int(best_cart['max_depth'])}")
plt.savefig(os.path.join(pathdic['results'], f'cart_tree_top{int(best_cart['top_n_features'])}_depth{int(best_cart['max_depth'])}.png'), dpi=300, bbox_inches='tight')
plt.close()

####generate predictions
y_pred = cart_model.predict(X_processed)
merged_glass['predicted_outcome'] = merged_glass['hip_fracture_count']
#apply predictions to df
merged_glass.loc[merged_glass['hip_fracture_count'].isna(), 'predicted_outcome'] = y_pred
###ADD GROUND TRUTH DATA TO PREDICTED
##save the df
merged_glass.to_csv(os.path.join(pathdic['results'], 'predictions_df.csv'))
pred_series = merged_glass['predicted_outcome']
##save outcomes to txt file
with open(os.path.join(pathdic['results'], 'cart_prediction.txt'), 'w') as f:
	for idx, val in pred_series.items():
		f.write(f"{idx}: {val}\n")
#X_u.head()


XGBoost model trained successfully
importance_df shape: (6, 4)
Number of features with importance: 6

Top features:
      Feature         Gain  Cumulative Gain  Cumulative Gain %
4       total  1655.493042      1655.493042          35.416351
3        male  1111.912964      2767.406006          59.203765
5  population   919.013672      3686.419678          78.864440
2      female   510.558655      4196.978333          89.786941
0   age_group   410.789062      4607.767395          98.575047
1      gender    66.607628      4674.375023         100.000000

Adjusted top_n_list from [7, 9, 13, 15] to [3, 4, 5, 6]
cart_df shape: (12, 7)
cart_df columns: ['label', 'top_n_features', 'max_depth', 'accuracy', 'precision', 'recall', 'specificity']

CART grid search completed. Generated 12 models.

Best efficient CART model:
label                  dfk
top_n_features           3
max_depth                5
accuracy          0.994852
precision              1.0
recall            0.984147
specificity   

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- age_group
- cold_lag
- cold_weather
- counties_in_region
- county_weight
- ...
